In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('100kRatings.csv')
df[:]

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
100955,757,55553,3.5,1196087036
100956,757,55820,5.0,1196086920
100957,757,55995,3.5,1196087242
100958,757,56145,4.0,1196115602


In [3]:
mn = df['movieId'].max()
df['movieId'].min()
mn

206272

In [4]:
groups = df.groupby('userId').groups
groups

{1: Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
             34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
             51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
             68, 69],
            dtype='int64'),
 2: Int64Index([ 70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
             ...
             244, 245, 246, 247, 248, 249, 250, 251, 252, 253],
            dtype='int64', length=184),
 3: Int64Index([254, 255, 256, 257, 258, 259, 260, 261, 262, 263,
             ...
             900, 901, 902, 903, 904, 905, 906, 907, 908, 909],
            dtype='int64', length=656),
 4: Int64Index([ 910,  911,  912,  913,  914,  915,  916,  917,  918,  919,
             ...
             1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151],
            dtype='int64', length=242),
 5: Int64Index([1152, 1153, 1154, 1155, 11

In [5]:
un = len(groups)

In [6]:
watched = np.zeros(mn + 1).astype(bool)
watched[df['movieId']] = True
mn2 = watched.sum()

In [7]:
ind = 0
dic = {}
dic_inv = {}
for i in range(mn + 1):
    if watched[i]:
        dic[i] = ind
        dic_inv[ind] = i
        ind += 1
ind

9949

In [8]:
mat = np.zeros([un, mn2])
mat.shape

(757, 9949)

In [9]:
dic[df['movieId'][30]]

3597

In [10]:
for i in range(un):
    for j in groups[i + 1]:
        mat[i][dic[df['movieId'][j]]] = (df['rating'][j] >= 3)*1.0
mat.dtype

dtype('float64')

In [11]:
df['movieId'][groups[i + 1]] - 1

99605         0
99606         1
99607        13
99608        15
99609        17
          ...  
100955    55552
100956    55819
100957    55994
100958    56144
100959    56787
Name: movieId, Length: 1355, dtype: int64

In [12]:
mat = mat.astype(int)
mat.shape

(757, 9949)

In [13]:
import csv
with open("mat.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(mat)
    f.close()

In [14]:
mat.shape

(757, 9949)

In [15]:
un

757

In [26]:
dic2 = {}
for i in dic:
    dic2[str(i)] = dic[i]

In [33]:
import pickle
f1 = open("dic", "wb")
pickle.dump(dic, f1)
f2 = open("dic_inv", "wb")
pickle.dump(dic_inv, f2)
f1.close()
f2.close()